In [1]:
from astropy.table import Table

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [2]:
data = Table.read('../fits/NSA/nsa_v0_1_2.fits')
zoo = Table.read('../fits/full_sample_debiased.fits')
data['index'] = np.arange(len(data))
zoo['index'] = np.arange(len(zoo))

In [3]:
# match the NSA to our debiased catalogue?
from astropy.coordinates import ICRS,match_coordinates_sky
from astropy import units as u

c = ICRS(data['RA'],data['DEC'], unit=(u.degree, u.degree))
catalog = ICRS(zoo['ra_1'],zoo['dec_1'], unit=(u.degree, u.degree))
idx, d2d, d3d = match_coordinates_sky(c, catalog)

reasonable_match = np.array(d2d) <= 20/3600 # 20 arcsec

In [10]:
data_matched = data[reasonable_match]
zoo_matched = zoo[idx][reasonable_match]
z_matched = np.abs(data_matched['Z']-zoo_matched['REDSHIFT_1']) <= 0.001
data_matched = data_matched[z_matched]
zoo_matched = zoo_matched[z_matched]
print('---> have {} matches between the NSA and GZ data!'.format(len(zoo_matched)))

---> have 49398 matches between the NSA and GZ data!


In [11]:
zoo_match = np.ones(len(data))*(-999)
zoo_match[data_matched['index'].astype(int)] = zoo_matched['index'].astype(int)
data['zoo_match'] = zoo_match
data.write('../fits/NSA/nsa_matched.fits',overwrite=True)